<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/loading_all_vector_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [0]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [0]:
!pip install -q 'canopy==7.2'

In [0]:
import canopy
import logging

logging.basicConfig(level=logging.INFO)

### Authenticate

In [0]:
authentication_data = canopy.prompt_for_authentication()

def create_session():
    return canopy.Session(authentication_data)

# Set Up Example

### Run a simulation using default configs

In [4]:
default_car_name = 'Canopy F1 Car 2019'
default_weather_name = '25 deg, dry'

async with create_session() as session:
   
    logging.info('Loading configs...')
    
    default_car = await canopy.load_default_config(session, 'car', default_car_name)
    default_weather = await canopy.load_default_config(session, 'weather', default_weather_name)

    logging.info('Creating study...')
    
    study_id = await canopy.create_study(
        session,
        'apexSim',
        'Python API Test Study',
        [
            default_car,
            default_weather,
        ])

    logging.info('Waiting for study to complete...')

    wait_result = await canopy.wait_for_study(
        session,
        study_id,
        timeout_seconds=60)

logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Succeded Simulations: {}/{}'.format(
    wait_result.succeeded_simulation_count, 
    wait_result.simulation_count))

INFO:root:Loading configs...
INFO:root:Creating study...
INFO:root:Waiting for study to complete...
INFO:root:Sim Version: 1.3043
INFO:root:Succeded Simulations: 1/1


# Example: Load all channel data
_Note:_ For performance reasons it is usually recommended to load only the channels you need. However loading all channels is possible by first fetching the channel list from the vector metadata.

In [5]:
async with create_session() as session:
    job_metadata = await canopy.load_study_job(
        session,
        study_id,
        'ApexSim',
        include_vector_metadata=True)
job_metadata.vector_metadata

INFO:canopy.load_study_job:Loading job index 0


,units,description,NPtsInChannel,xDomainName
name,,,,
xCar,m,x position of the car centre of gravity in the...,42,vCar
yCar,m,y position of the car centre of gravity in the...,42,vCar
zCar,m,z position of the car centre of gravity in the...,42,vCar
aRoll,rad,Roll angle of the car in the world coordinate ...,42,vCar
aPitch,rad,Pitch angle of the car in the world coordinate...,42,vCar
...,...,...,...,...
aSlipTyreR,rad,Average slip angle of the rear tyres.,42,vCar
zTrackFL,m,Track height at the front left contact patch.,42,vCar
zTrackFR,m,Track height at the front right contact patch.,42,vCar


The vector_metadata is a DataFrame indexed by the channel name, so we can extract the list of channels by asking for the index column.


In [6]:
all_channels = job_metadata.vector_metadata.index
all_channels

Index(['xCar', 'yCar', 'zCar', 'aRoll', 'aPitch', 'aYaw', 'vCar', 'dxCar',
       'dyCar', 'dzCar',
       ...
       'nWheelRR', 'dnWheelRR', 'aSteerKinematicRR', 'aSteerTotalRR',
       'aSlipTyreF', 'aSlipTyreR', 'zTrackFL', 'zTrackFR', 'zTrackRL',
       'zTrackRR'],
      dtype='object', name='name', length=382)

We can now pass that list back into the `load_study_job` function to load all the channel data.

In [7]:
async with create_session() as session:
    job_data = await canopy.load_study_job(
        session,
        study_id,
        'ApexSim',
        channel_names=all_channels)

logging.info('Loaded {} channels'.format(len(job_data.vector_data.columns)))

INFO:canopy.load_study_job:Loading job index 0
INFO:root:Loaded 382 channels


In [8]:
job_data.vector_data

,xCar,yCar,zCar,aRoll,aPitch,aYaw,vCar,dxCar,dyCar,dzCar,nRoll,nPitch,nYaw,gLong,gLat,gVert,dnRoll,dnPitch,dnYaw,aSlipCar,aSteerWheel,aSteerL,aSteerR,gLongAccelerometer,gLatAccelerometer,gVertAccelerometer,xRackL,xRackR,d2xRackL,d2xRackR,FRackL,FRackR,hRideF,hRideR,aInGGPlane,rLatBalF,cUnderOverSteer,PEnergyKinetic,hRollCentreF,hRollCentreR,...,dFyTyre_daSlipRL,MyRollingResistanceRL,nWheelRL,dnWheelRL,aSteerKinematicRL,aSteerTotalRL,aSlipTyreRR,rSlipTyreRR,aCamberTyreRR,rTyreLoadedRR,xTyreSquashRR,rTyreRollingRadiusRR,xBumpRR,kHeaveTyreRR,PTyreWearRR,PTyreWearLatRR,PTyreWearLongRR,rGripTyreWearRR,rGripTyreRR,rGripTemperatureRR,FxTyreRR,FyTyreRR,FzTyreRR,MxTyreRR,MyTyreRR,MzTyreRR,FxyTyreRR,dFxTyre_drSlipRR,dFyTyre_daSlipRR,MyRollingResistanceRR,nWheelRR,dnWheelRR,aSteerKinematicRR,aSteerTotalRR,aSlipTyreF,aSlipTyreR,zTrackFL,zTrackFR,zTrackRL,zTrackRR
0,0.0,0.0,-0.329210,-0.013374,-0.028219,0.0,9.894445,9.898387,1.421950,0.0,0.0,0.0,1.115020,-3.336827,10.782159,0.238409,0.016614,0.035068,4.209901e-08,0.143092,4.861905,0.419795,0.405676,-3.613619,10.913305,-9.566808,-0.038134,0.038538,1.780097e-09,4.041765e-10,-5557.942871,1520.608276,0.023581,0.125156,1.870924,0.330607,0.003171,-2.455205e+04,0.051553,0.031666,...,62577.132812,1.373588,31.928099,-5.854144,0.003029,0.003029,-0.039399,-0.003411,-0.023205,0.331409,0.003591,0.335022,0.053972,237007.484375,297.923248,295.627808,2.295421,1.0,1.0,1.0,-74.643188,832.305664,-878.518372,-154.231888,0.0,-43.399925,835.646057,23142.226562,17499.906250,0.324085,26.909155,-14.074288,-0.003288,-0.003288,-0.066443,-0.039141,0.0,0.0,0.0,0.0
1,0.0,0.0,-0.328919,-0.014789,-0.028291,0.0,11.929656,11.934432,1.252730,0.0,0.0,0.0,1.170615,-3.914644,13.709241,0.313557,0.020250,0.038751,-7.508502e-09,0.105029,4.685944,0.401159,0.391341,-4.192139,13.854261,-9.491446,-0.036555,0.037095,-5.492089e-07,1.295209e-07,-7349.637695,1949.445068,0.023154,0.124987,1.848942,0.408030,0.012541,-3.473238e+04,0.053721,0.032957,...,50524.976562,1.719215,38.090752,-6.580427,0.003019,0.003019,-0.055035,-0.003866,-0.024567,0.331823,0.003177,0.335032,0.054478,236257.046875,581.621887,578.588806,3.033113,1.0,1.0,1.0,-71.345543,955.942017,-783.018799,-145.372177,0.0,-45.136429,958.600708,20561.976562,12036.092773,0.351170,32.825783,-16.798281,-0.003302,-0.003302,-0.111064,-0.054099,0.0,0.0,0.0,0.0
2,0.0,0.0,-0.328391,-0.013976,-0.028382,0.0,13.964086,13.969712,0.920404,0.0,0.0,0.0,1.074217,-3.809530,14.816415,0.315249,0.016114,0.032737,1.578938e-08,0.066206,3.787462,0.317802,0.320089,-4.087924,14.953459,-9.489842,-0.029319,0.029987,-2.726487e-07,3.028698e-08,-8205.988281,1530.829346,0.022443,0.124606,1.822461,0.473971,0.011882,-3.956642e+04,0.054757,0.032078,...,44932.808594,1.971324,43.853703,-6.769616,0.003023,0.003023,-0.058982,-0.008945,-0.023809,0.331356,0.003644,0.335045,0.053937,237124.406250,902.746521,881.075256,21.671263,1.0,1.0,1.0,-185.896561,1146.205811,-896.676453,-157.729874,0.0,-76.821800,1161.182739,23334.382812,12730.049805,0.478405,38.897720,-15.980394,-0.003288,-0.003288,-0.108912,-0.058662,0.0,0.0,0.0,0.0
3,0.0,0.0,-0.327894,-0.013502,-0.028428,0.0,15.981848,15.988308,0.611572,0.0,0.0,0.0,0.984569,-3.803468,15.614928,0.319001,0.013077,0.027545,3.393705e-08,0.038628,3.178587,0.261705,0.271101,-4.082305,15.747321,-9.486142,-0.024394,0.025163,4.134776e-10,9.563485e-12,-9005.409180,1311.043579,0.021855,0.124181,1.809723,0.513248,0.012477,-4.521364e+04,0.055274,0.031509,...,41837.636719,2.240719,49.577263,-7.116450,0.003023,0.003023,-0.061110,-0.012619,-0.023378,0.330984,0.004016,0.335060,0.053517,237622.593750,1240.812378,1186.555908,54.256470,1.0,1.0,1.0,-285.763550,1290.510376,-988.433899,-165.784164,0.0,-102.916069,1321.770752,25403.761719,13401.686523,0.610154,44.904343,-15.596267,-0.003278,-0.003278,-0.111661,-0.061352,0.0,0.0,0.0,0.0
4,0.0,0.0,-0.327381,-0.013282,-0.028453,0.0,17.989611,17.996895,0.334298,0.0,0.0,0.0,0.910603,-3.881660,16.317278,0.327225,0.011004,0.023582,2.276238e-08,0.018957,2.748405,

Once these are loaded we can request individual channels in specific units as per usual.

In [10]:
job_data.vector_as('hRideF', 'mm')

0     23.580887
1     23.154284
2     22.442993
3     21.855398
4     21.292150
5     20.719418
6     20.141134
7     19.575432
8     19.167400
9     18.525843
10    17.840559
11    17.123169
12    16.377218
13    15.614468
14    14.846750
15    14.149844
16    13.395889
17    12.593701
18    11.775322
19    10.903580
20    10.008492
21     9.097126
22     8.213157
23     7.244925
24     6.251575
25     5.233883
26     4.193030
27     3.165147
28     2.093131
29     0.985856
30    -0.145208
31    -1.302221
32    -2.460294
33    -3.669101
34    -4.907894
35    -6.176690
36    -7.451297
37    -8.755168
38    -9.865002
39   -10.360962
40   -10.722709
41   -11.073380
Name: hRideF, dtype: float32